# Directory

> directory class.

In [ ]:
#| default_exp utils.directory

In [ ]:
#| hide
from nbdev.showdoc import *

In [113]:
#| export
import os, pathlib 
from pathlib import Path
from dataclasses import dataclass, field, KW_ONLY
from typing import Optional, List, ClassVar, Any, TypeAlias
from enum import StrEnum

In [88]:
#| export
from iza.types import PathLike, PathType
from iza.static import EXT_PY

### Directory Viewer

#### Directory

In [89]:
#| export
class DirectoryTreeStrings(StrEnum):
    SPACE : ClassVar[str] = '    '
    BRANCH: ClassVar[str] = '│   '    
    TEE   : ClassVar[str] = '├── '
    LAST  : ClassVar[str] = '└── '

In [90]:
#| export
class ExtensionIcons(StrEnum):
    PY: ClassVar[str] = "🐍"

    @classmethod    
    def get(cls, path:PathLike):
        if isinstance(path, str): 
            path = Path(path)
            if path.suffix == EXT_PY:
                return cls[EXT_PY.lstrip('.').upper()]
    

In [114]:
#| export
try:    
    from rich.tree import Tree
    from rich.text import Text
    from rich.markup import escape
    from rich.filesize import decimal
    from rich import get_console
    from rich.console import Console
    from rich.progress import Progress
    TreeType = Tree
    TextType = Text    
    ConsoleType = Console
    ProgressType = Progress

    def rich_link_style(path:Path):
        return f'link file://{path}'
    
    def rich_link_file(path: Path):
        return f'[{rich_link_style(path)}]{escape(path.name)}[/]'
    
    def rich_file_size(path: Path):
        size = path.stat().st_size
        return decimal(size)
    
    def rich_file(path: Path):
        size = rich_file_size(path)
        text = Text(path.name)
        text.stylize(rich_link_style(path))
        text.append(f' ({size})', "cyan")
        return text
    
except ImportError:
    Tree = None
    Text = None 
    TreeType = Any
    TextType = Any
    ConsoleType = Any
    Progress = None
    ProgressType = Any
    
    def rich_file(path: Path):
        return path.name

In [ ]:
#| export
def dir_tree_entry(path:Path, pointer:Optional[str]='', prefix:Optional[str]='') -> str:
    name = rich_file(path)
    # NOTE: defined in _02_utils/_08_modules.ipynb
    if is_rich_available():
        return name
    return f'{prefix}{pointer}{path.name}'

#| export
def try_init_rich_tree(dirname:PathLike) -> Optional[TreeType]:
    if is_rich_available() and 'Tree' in globals():
        return Tree(f'[link file://{dirname}]{dirname}', guide_style='bold bright_blue')
    return None

In [127]:
#| export
def walk_dir_tree(
    dirname: PathLike, prefix: str = '',
    hidden: Optional[bool] = False, tree: Optional[TreeType] = None,
):    
    '''
    A recursive generator, given a directory Path object
    will yield a visual tree structure line by line
    with each line prefixed by the same characters
    Notes
    -----
    Adapted from https://stackoverflow.com/a/59109706/5623899
    '''
    # NOTE: sort_directory_first, to_abs_expanded, rich_file defined in _02_utils/_08_modules.ipynb
    dirname = Path(to_abs_expanded(dirname))
    contents = sorted(Path(dirname).iterdir(), key=sort_directory_first)
    
    SPACE, BRANCH, TEE, LAST = DirectoryTreeStrings
    pointers = [TEE] * (len(contents) - 1) + [LAST]
    
    for pointer, path in zip(pointers, contents):
        # Remove hidden files
        if path.name.startswith('.') and not hidden:
            continue
        
        name = dir_tree_entry(path, pointer, prefix)
        branch = None
        if tree is not None:
            branch = tree.add(rich_file(path))
        yield name

    
        if path.is_dir():
            # NOTE: space because last, └── , above so no more |
            extension = BRANCH if pointer == TEE else SPACE
            yield from walk_dir_tree(path, prefix=f'{prefix}{extension}', hidden=hidden, tree=branch)

In [128]:
#| export
@dataclass
class DirectoryTree:
    dirname: str
    hidden: Optional[bool] = False
    def __post_init__(self):
        # NOTE: defined in _02_utils/_01_files.ipynb
        self.dirname = Path(to_abs_expanded(self.dirname))

    def tree_generator(self, dirname: Optional[PathLike] = None):
        dirname = getattr(self, 'dirname', dirname)
        tree = getattr(self, 'tree', None)
        yield from walk_dir_tree(self.dirname, prefix='', hidden=self.hidden, tree=tree)
        self.tree = tree
        
    def get_tree_lines(self, dirname: Optional[PathLike] = None) -> List[str]:
        dirname = getattr(self, 'dirname', dirname)
        tree_gen = self.tree_generator(dirname)
        lines = [line for line in tree_gen]
        return lines

In [ ]:
#| export
@dataclass
class RichTreeMixin(DirectoryTree):
    console: Optional[ConsoleType] = field(default_factory=Console, init=False, repr=False)
    tree: Optional[TreeType] = None
    
    def __post_init__(self):
        super().__post_init__()
        if self.console is None and is_rich_available():
            self.console = get_console()

        if self.tree is None:
            self.tree = try_init_rich_tree(self.dirname)

    def print_rich(self, dirname: Optional[str] = None, tree: Optional[TreeType] = None) -> None:
        tree = getattr(self, 'tree', tree)
        if tree is None:
            self.tree = try_init_rich_tree(self.dirname)

        lines = self.get_tree_lines(dirname)
        self.console.print(self.tree)

In [25]:
#| export 
@dataclass
class Directory(RichTreeMixin):
    dirname: str
    
    def __post_init__(self):
        super().__post_init__()

    def make_tree_str(self, dirname: Optional[PathLike] = None) -> str:
        dirname = getattr(self, 'dirname', dirname)
        lines = self.get_tree_lines(dirname)
        tree_str = '\n'.join([str(dirname), *lines])
        return tree_str
   
    def print(self, dirname:Optional[str]=None) -> None:
        dirname = self.prepare_dirname(dirname)
        tree_str = self.make_tree_str(dirname)        
        print(tree_str)
        return

    def __repr__(self):
        dirname = getattr(self, 'dirname', None)
        tree_str = self.make_tree_str(dirname)        
        return tree_str        

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()